# Problem Set 6: Text Analysis of DOJ Press Releases, Part 1

- For background:
    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 

0.0 Import packages

In [31]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## Install relevant packages if you haven't already
! conda activate qss20
! conda install nltk spacy vaderSentiment gensim scikit-learn "scipy<1.13"

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## LDA
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

Channels:
 - conda-forge
Platform: win-64
Solving environment: done

# All requested packages already installed.



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sharp\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sharp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------- ------------------------------ 3.1/12.8 MB 20.6 MB/s eta 0:00:01
     ------------------------ --------------- 7.9/12.8 MB 22.1 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 22.5 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 21.7 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


0.1 Load and clean text data

In [32]:
## first, unzip the file pset6_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("pset6_inputdata/combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [33]:
## your code to subset to one press release and take the string

pharma = doj[doj["id"] == "17-1204"]["contents"]

print(pharma)

4909    The founder and majority owner of Insys Therapeutics Inc., was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain. "More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. And yet some medical professionals would rather take advantage of the addicts than try to help them," said Attorney General Jeff Sessions. "This Justice Department will not tolerate this.  We will hold accountable anyone – from street dealers to corporate executives -- who illegally contributes to this nationwide epidemic.  And under the leadership of President Trump, we are fully committed to defeating this threat to the American people.”John N. Kapoor, 74, of Phoenix, Ariz., a current member of the Board of Directors of Insys, was arrested this morning in Arizona and charged with RICO conspira

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [34]:
## your code here to restrict to alpha
pharma_alpha = " ".join([word for word in pharma.iloc[0].split() if word.isalpha()])

def punc_dig_check(text):
    if any(char in string.punctuation for char in text):
        print("Punctuation still exists in the string.")
    else:
        print("No punctuation found in the string.")

    if any(char in string.digits for char in text):
        print("Digits still exist in the string.")
    else:    
        print("No digits found in the string.")

punc_dig_check(pharma_alpha)
print("\nContent of pharma_alpha:\n" + pharma_alpha)


No punctuation found in the string.
No digits found in the string.

Content of pharma_alpha:
The founder and majority owner of Insys Therapeutics was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough than Americans died of synthetic opioid overdoses last and millions are addicted to And yet some medical professionals would rather take advantage of the addicts than try to help said Attorney General Jeff Justice Department will not tolerate We will hold accountable anyone from street dealers to corporate executives who illegally contributes to this nationwide And under the leadership of President we are fully committed to defeating this threat to the American of a current member of the Board of Directors of was arrested this morning in Arizona and charged with RICO as well as other including conspiracy to commit mail and wire fraud

In [35]:
## your code here for part of speech tagging
tokens = wordpunct_tokenize(pharma_alpha)
tagged = pos_tag(tokens)

print(tokens[:10])
print(tagged[:10])

['The', 'founder', 'and', 'majority', 'owner', 'of', 'Insys', 'Therapeutics', 'was', 'arrested']
[('The', 'DT'), ('founder', 'NN'), ('and', 'CC'), ('majority', 'NN'), ('owner', 'NN'), ('of', 'IN'), ('Insys', 'NNP'), ('Therapeutics', 'NNP'), ('was', 'VBD'), ('arrested', 'VBN')]


## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [36]:
## your code here for part A
pharma_text = pharma.iloc[0] if isinstance(pharma, pd.Series) else str(pharma)
doc = nlp(pharma_text)

ents = [(ent.text, ent.label_) for ent in doc.ents]
ents_df = pd.DataFrame(ents, columns=["entity", "label"])
ents_df["count_in_doc"] = ents_df.groupby("entity")["entity"].transform("count")
ents_df = ents_df.drop_duplicates().sort_values(["label", "count_in_doc"], ascending=[True, False])

print("Top 10 entities in the document:")
ents_df.head(10)

Top 10 entities in the document:


,entity,label,count_in_doc
3,"More than 20,000",CARDINAL,1
6,millions,CARDINAL,1
61,six,CARDINAL,1
1,today,DATE,3
65,Today,DATE,2
111,three years,DATE,2
5,last year,DATE,1
12,74,DATE,1
28,a later date,DATE,1
32,December 2016.The,DATE,1


In [37]:
## your code here for part B
law_ents = sorted({ent.text for ent in doc.ents if ent.label_ == "LAW"})
print("Unique LAW entities:")
for e in law_ents:
    print("-", e)

Unique LAW entities:
- RICO
- the Controlled Substances Act


C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

In [38]:
## your code here
rico_summary = ("RICO is a U.S. law that lets prosecutors charge people who participate in an ongoing criminal enterprise, so it can apply to a pharmaceutical kickback scheme if the conduct is organized and repeated through a coordinated network, not just to mafia cases.")

print(rico_summary)

RICO is a U.S. law that lets prosecutors charge people who participate in an ongoing criminal enterprise, so it can apply to a pharmaceutical kickback scheme if the conduct is organized and repeated through a coordinated network, not just to mafia cases.


D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [39]:
## your code here
date_year_ents = []
for ent in doc.ents:
    if ent.label_ == "DATE" and re.search(r"\byears?\b", ent.text):
        date_year_ents.append(ent.text)

date_year_ents = sorted(set(date_year_ents))
print("DATE entities that mention years:")
for e in date_year_ents:
    print("-", e)

DATE entities that mention years:
- 20 years
- five years
- last year
- three years


E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [40]:
## your code here
sentences = re.split(r'(?<=[.!?])\s+', pharma_text.strip())

# Sentences that mention year(s) OR a number + year(s)
year_sents = [s for s in sentences if re.search(r"(\b\d+\s+years?\b|\byears?\b)", s)]
print("Sentences mentioning year lengths:")
for s in year_sents:
    print("-", s)

# helper func to get all int years from a sentence
def extract_year_nums(s):
    return [int(x) for x in re.findall(r"\b(\d+)\s+years?\b", s)]

# Classify sentences into prison vs. supervised release
prison_years = []
sr_years = []

for s in year_sents:
    yrs = extract_year_nums(s)
    if not yrs:
        continue
    # prison cues
    if re.search(r"imprisonment|prison|incarcerat|sentenc", s):
        prison_years += yrs
    # supervised release / probation cues
    if re.search(r"supervised release|probation", s):
        sr_years += yrs

max_prison = max(prison_years) if prison_years else None
max_sr = max(sr_years) if sr_years else None

summary = "Based on the press release, "
if max_prison is not None:
    summary += f"the maximum prison exposure mentioned is {max_prison} years"
else:
    summary += "the press release does not clearly state a maximum prison term in years"
if max_sr is not None:
    summary += f", and the maximum supervised release/probation term mentioned is {max_sr} years."
else:
    summary += ", and it does not clearly state a supervised release term in years."

print("\nInterpretation (1 sentence):")
print(summary)



Sentences mentioning year lengths:
- "More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids.
- Neves, Special Agent in Charge of the VA OIG Northeast Field Office.The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.
- The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine.

Interpretation (1 sentence):
Based on the press release, the maximum prison exposure mentioned is 20 years, and the maximum supervised release/probation term mentioned is 20 years.


## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [41]:
## your code here for subsetting

topics_keep = ["Civil Rights", "Hate Crimes", "Project Safe Childhood"]

# Format check. Delimited by ;
# for x in doj["topics_clean"].dropna().unique():
    # print(x)

# Only keep rows with exactly one topic (no semicolon) and that topic is in topics_keep
single_topic_mask = doj["topics_clean"].notna() & (doj["topics_clean"].str.count(";") == 0)
in_keep_mask = doj["topics_clean"].isin(topics_keep)
mask = single_topic_mask & in_keep_mask     # Boolean series that is True for rows we want to keep

doj_subset = doj.loc[mask].copy()
doj_subset["topic_group"] = doj_subset["topics_clean"].str.strip()

print("Rows in doj_subset:", doj_subset.shape[0])
doj_subset[["id", "topic_group", "topics_clean"]].head()

Rows in doj_subset: 717


,id,topic_group,topics_clean
77,17-1235,Civil Rights,Civil Rights
155,15-1522,Project Safe Childhood,Project Safe Childhood
157,16-213,Project Safe Childhood,Project Safe Childhood
162,16-381,Project Safe Childhood,Project Safe Childhood
168,14-464,Hate Crimes,Hate Crimes


B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [42]:
## your code here to define function
sia = SentimentIntensityAnalyzer()

def sentiment_remove_ents(text, nlp):
    text = "" if text is None else str(text)
                                       
    doc = nlp(text)
    ent_texts = {ent.text.strip() for ent in doc.ents if ent.text.strip()}

    if ent_texts:
        ent_list = sorted(ent_texts, key=len, reverse=True)

        pattern = r"\b(" + "|".join(re.escape(ent) for ent in ent_list) + r")\b"

        text_no_ents = re.sub(pattern, " ", text)
        text_no_ents = re.sub(r"\s+", " ", text_no_ents).strip()

    else:
        text_no_ents = text

    return sia.polarity_scores(text_no_ents)

In [43]:
## your code here executing the function
scores = [sentiment_remove_ents(t, nlp) for t in doj_subset["contents"]]
scores_df = pd.DataFrame(scores)

doj_subset_wscore = pd.concat(
    [doj_subset.reset_index(drop=True), scores_df.reset_index(drop=True)],
    axis=1
)

doj_subset_wscore.head()

,id,title,contents,date,topics_clean,components_clean,topic_group,neg,neu,pos,compound
0,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle",Civil Rights,0.191,0.759,0.050,-0.9920
1,15-1522,Alabama Man Found Guilty of Aggravated Sexual Abuse of a Child,"A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced. According to evidence introduced at trial, Evans, a former U.S. Army soldier, and his then-wife, a Department of Defense employee, were residing in Germany when they were asked to take temporary custody of a five-year-old child whose parents were deployed to Iraq with the U.S. Army. Evans sexually abused the child on multiple occasions during the 18 months that the child lived with him from May 2007 to December 2008. Trial Attorney Austin M. Berry of the Criminal Division’s Child Exploitation and Obscenity Section (CEOS) and Assistant U.S. Attorney Jacquelyn Hutzell of the Northern District of Alabama are prosecuting the case. U.S. Army Criminal Investigations Division and the FBI’s Birmingham, Alabama, Division investigated the case. This case was brought as part of Project Safe Childhood, a nationwide initiative to combat the growing epidemic of child sexual exploitation and abuse, launched in May 2006 by the Department of Justice. Led by U.S. Attorneys’ offices and CEOS, Project Safe Childhood marshals federal, state and local resources to better locate, apprehend and prosecute individuals who exploit children via the Internet, as well as to identify and rescue victims. For more information about Project Safe Childhood, please visi

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [44]:
## your code here
doj_subset_wscore = pd.concat([doj_subset.reset_index(drop=True), scores_df.reset_index(drop=True)], axis=1)

doj_subset_wscore.sort_values("neg", ascending=False)[["id", "contents", "neg"]].head(10)

,id,contents,neg
13,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.320
632,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",0.296
34,13-312,"John Hall, 27, an Aryan Brotherhood member and inmate at the Federal Correctional Institution (FCI) in Seagoville, Texas, was sentenced today by U.S. District Judge Reed O’Connor after pleading guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act stemming from his assault of a fellow inmate, whom he believed to be gay, the Department of Justice announced. Hall assaulted his fellow inmate with a dangerous weapon, causing bodily injury to the victim on Dec. 20, 2011. Hall was sentenced to serve 71 months in prison to be served consecutively with the sentence he is currently serving. The assault occurred on Dec. 20, 2011, inside the FCI Seagoville when Hall targeted and attacked the victim, a fellow inmate, because he believed the victim was gay or involved in a sexual relationship with another male inmate. Hall repeatedly punched, kicked and stomped on the victim’s face with his shod feet, a dangerou

D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [45]:
## agg and find the mean compound score by topic
mean_compound = doj_subset_wscore.groupby("topic_group").agg(mean_compound = ("compound", "mean")).sort_values("mean_compound")

mean_compound

,mean_compound
topic_group,
Hate Crimes,-0.929995
Project Safe Childhood,-0.688322
Civil Rights,-0.098452


# 2. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [46]:
# your code here 

# Text preprocessing function (turn lowercase + strip punctuation and digits)
def simple_clean(s):
    s = str(s).lower()
    s = re.sub(r"[^a-z\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# Convert to list of cleaned docs, and get ids and titles in lists too
docs = doj_subset["contents"].map(simple_clean).tolist()
ids = doj_subset["id"].tolist()
titles = doj_subset["title"].tolist()
contents = doj_subset["contents"].tolist()

# Print first 2 cleaned docs to check
# print(docs [:2])  

# Vectorize and remove common english stop words, only keep words that appear in >=2 docs, drop words that appear >95% of docs
cv = CountVectorizer(stop_words="english", min_df=2, max_df=0.95, binary=True)
X = cv.fit_transform(docs)

# Dot products (sparse)
dot = (X @ X.T).toarray()   # make dense similarity matrix

# Row norms
# ||x||^2 for each row
row_sq = np.asarray(X.power(2).sum(axis=1)).ravel()
norms = np.sqrt(row_sq)

best = []  # store (sim, i, j)

for i in range(X.shape[0]):
    # Dot products of doc i with docs i+1 to end (intersection counts)
    dots = (X[i] @ X[i+1:].T).toarray().ravel()

    # Cosine similarity with docs i+1 to end
    sims = dots / (norms[i] * norms[i+1:] + 1e-12)

    # Keep top 2 from this row
    if sims.size > 0:
        k = min(2, sims.size)
        top_js = np.argpartition(sims, -k)[-k:]
        for idx_j in top_js:
            j = i + 1 + idx_j
            best.append((sims[idx_j], i, j))

# Global top 2
best = sorted(best, reverse=True)[:2]

for rank, (sim, i, j) in enumerate(best, start=1):
    print(f"\nPair #{rank} (cosine similarity = {sim:.3f})")
    print(f"- A: {ids[i]} | {titles[i]}")
    print(f"- B: {ids[j]} | {titles[j]}")
    print("\nA snippet:")
    print(contents[i][:350].replace("\n"," ") + " ...")
    print("\nB snippet:")
    print(contents[j][:350].replace("\n"," ") + " ...")


Pair #1 (cosine similarity = 0.967)
- A: 17-815 | Member of International Child Exploitation Conspiracy Sentenced to 210 Months in Prison
- B: 17-782 | Member of International Child Exploitation Conspiracy Sentenced to 210 Months in Prison

A snippet:
A Wichita, Kansas man was sentenced to 210 months in prison and 10 years of supervised release, for production of child pornography based on his participation in a website that was operated for the purpose of coercing and enticing minors as young as eight years old to engage in sexually explicit conduct on web camera.   Acting Assistant Attorney Ge ...

B snippet:
A Chicopee, Massachusetts man was sentenced to 210 months in prison and 10 years of supervised release for production of child pornography based on his participation in a website that was operated for the purpose of coercing and enticing minors as young as eight years old to engage in sexually explicit conduct on web camera.  Acting Assistant Attor ...

Pair #2 (cosine similari

Pair #1 appears to be two separate press releases about different defendants involved in the same international child exploitation conspiracy, using nearly identical DOJ boilerplate language and the same sentence length. Pair #2 looks like two stages of the same Virginia case, with one release reporting the conviction and the other reporting the later sentencing for the same underlying crimes.